### iris 데이터를 활용하여 그리드 서치를 적용해보자

데이터 로드 및 분리

In [10]:
from sklearn.neighbors import KNeighborsClassifier # knn 모델
from sklearn.tree import DecisionTreeClassifier # 의사결정나무

# 그리드 서치
from sklearn.model_selection import GridSearchCV

# 데이터 분할 도구
from sklearn.model_selection import train_test_split

# iris 데이터
from sklearn.datasets import load_iris

In [18]:
# iris 데이터 로드
iris = load_iris()

X_train, X_test, y_train, y_test = train_test_split(
    iris['data'],
    iris['target'],
    test_size=0.3,
    random_state=8
)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(105, 4)
(45, 4)
(105,)
(45,)


### KNN모델에 그리드 서치 적용

In [27]:
param_knn = {'n_neighbors': range(1, 50, 1)}
# 딕셔너리 형태
# key 값에는 하이퍼파라미터명, value 값에는 찾고자하는 값의 범위

# GridSearchCV
grid_search_knn = GridSearchCV(
    KNeighborsClassifier(), # knn 모델에서
    param_knn, # 찾고자하는 하이퍼파라미터(key 값)와 범위(value 값)
    cv=10 # 교차검증
)

grid_search_knn.fit(X_train, y_train) # 최고성능모델이 변수에 저장

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 50)})

- GridSearchCV는 학습한 모델의 최적의 하이퍼파라미터값, 점수, 모델명 속성을 통해 확인

In [34]:
print('최적의 파라미터 값: ', grid_search_knn.best_params_)
print('최고 점수: ', grid_search_knn.best_score_)
print('최고 성능 모델: ', grid_search_knn.best_estimator_)

최적의 파라미터 값:  {'n_neighbors': 3}
최고 점수:  0.9809090909090911
최고 성능 모델:  KNeighborsClassifier(n_neighbors=3)


### 정리
- 이전에 구한 최적의 하이퍼파라미터 k=7과 다름
- 이전에는 교차검증도 진행하지 않았으므로, 랜덤 스플릿된 데이터셋 자체에 대한 과대적합의 위험이 있었음
- 차이점: 교차검증 진행 여부
- grid search를 통한 k=3값은 10번의 교차검증을 마쳤을 때, 10개의 정확도의 평균이 가장 높은 k값을 찾음

In [41]:
# test로 평가
grid_search_knn.score(X_test, y_test)

# 전보다(k=7일때보다) 낮게 나옴
# 현재 random_state=8
# random_state=8 이라면, k가 7일 때 가장 높게 나옴
# random_state!=8, 평균적으로 k=3일때 정확도가 높음(교차검증을 통해)

0.9333333333333333

### 결정트리 모델의 최적의 하이퍼파라미터
- max_depth, max_leaf_node, min_samples_leaf 값들을 설정하여 최적값을 찾아보자

In [48]:
# key값에는 하이퍼파라미터명, value 값에는 찾고자하는 그 범위 지정
param_tree = {
    'max_depth': range(1, 20, 2), # 1, 3, 5, 7,..., 19 (총 10개)
    'max_leaf_nodes': range(10, 101, 10), # 10, 20, 30,..., 100 (총 10개)
    'min_samples_leaf': range(20, 151, 10) # 20, 30, 40,..., 150 (총 14개)
}

# GridSearchCV 함수 선언
grid_search_tree = GridSearchCV(
    DecisionTreeClassifier(),
    param_tree, # 찾고자하는 하이퍼파라미터와 그 범위
    cv=7,
    verbose=2 # 진행상태 표시
)

grid_search_tree.fit(X_train, y_train)

Fitting 7 folds for each of 1400 candidates, totalling 9800 fits
[CV] END max_depth=1, max_leaf_nodes=10, min_samples_leaf=20; total time=   0.0s
[CV] END max_depth=1, max_leaf_nodes=10, min_samples_leaf=20; total time=   0.0s
[CV] END max_depth=1, max_leaf_nodes=10, min_samples_leaf=20; total time=   0.0s
[CV] END max_depth=1, max_leaf_nodes=10, min_samples_leaf=20; total time=   0.0s
[CV] END max_depth=1, max_leaf_nodes=10, min_samples_leaf=20; total time=   0.0s
[CV] END max_depth=1, max_leaf_nodes=10, min_samples_leaf=20; total time=   0.0s
[CV] END max_depth=1, max_leaf_nodes=10, min_samples_leaf=20; total time=   0.0s
[CV] END max_depth=1, max_leaf_nodes=10, min_samples_leaf=30; total time=   0.0s
[CV] END max_depth=1, max_leaf_nodes=10, min_samples_leaf=30; total time=   0.0s
[CV] END max_depth=1, max_leaf_nodes=10, min_samples_leaf=30; total time=   0.0s
[CV] END max_depth=1, max_leaf_nodes=10, min_samples_leaf=30; total time=   0.0s
[CV] END max_depth=1, max_leaf_nodes=10, min

/opt/anaconda3/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=7, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 20, 2),
                         'max_leaf_nodes': range(10, 101, 10),
                         'min_samples_leaf': range(20, 151, 10)},
             verbose=2)

In [50]:
print('최적의 파라미터 값: ', grid_search_tree.best_params_)
print('최고 점수: ', grid_search_tree.best_score_)
print('최고 성능 모델: ', grid_search_tree.best_estimator_)

최적의 파라미터 값:  {'max_depth': 3, 'max_leaf_nodes': 10, 'min_samples_leaf': 20}
최고 점수:  0.980952380952381
최고 성능 모델:  DecisionTreeClassifier(max_depth=3, max_leaf_nodes=10, min_samples_leaf=20)


In [52]:
grid_search_tree.score(X_test, y_test)

0.8888888888888888

In [54]:
# knn 모델의 훈련데이터에서의 정확도
grid_search_knn.score(X_train, y_train)

0.9714285714285714

In [56]:
# knn 모델의 테스트데이터에서의 정확도
grid_search_knn.score(X_test, y_test)

0.9333333333333333

In [58]:
# tree 모델의 훈련데이터에서의 정확도
grid_search_tree.score(X_train, y_train)

0.9904761904761905

In [60]:
# tree 모델의 테스트데이터에서의 정확도
grid_search_tree.score(X_test, y_test)

0.8888888888888888